Dependencies

In [ ]:
import tensorflow as tensor
import numpy as np
import random

Get Train file

In [3]:

fpath = tensor.keras.utils.get_file('shakespeare.txt',
                                    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
ftext = open(fpath,"rb").read().decode(encoding="utf-8").lower()


1115394/1115394 [==============================] - 0s 0us/step


open : open file and return stream , rb : r:open for reading ;b: retrieve in binary mode
read : return file content, para default = -1 => whole file
decode: decode the UTF-8 stream into readable string
lower :change test to lowercase ( for simplicity)

Data Prep

In [12]:

# train_text =ftext[300000:800000]
#Use certain subset of data to train
char = sorted(set(ftext))
#set:  no repetition , unchangeable
char_index = dict((c,i) for i, c in enumerate(char)) # Character to index
index_char = dict((i,c) for i,c in enumerate(char)) # index to Character

SEQ_LEN = 40
STEP_SIZE = 3 # next sentence
sentences= [] # list of sentences
next_char=[] # next char in relation to  index of sentences

#store into lists
for i  in range(0,len(ftext)-SEQ_LEN,STEP_SIZE):
    sentences.append(ftext[i:i+SEQ_LEN])
    next_char.append(ftext[i+SEQ_LEN])

x = np.zeros((len(sentences),SEQ_LEN,len(char)),dtype=bool)
#zeros: array of zeros , 3-dim , data type = bool
y = np.zeros((len(sentences), len(char)),dtype = bool)
# 2-dim

for i, s in enumerate(sentences):
    for j,c in enumerate(s):
        x[i,j,char_index[c]] = 1 # 3_Dim array to represent whether a character is present; array: [sentence number ,  character position in sentence, which unique charcter it is from the char array ]
    y[i,char_index[next_char[i]]] = 1 # 2-dim  array to relate the sentence and the next character



Recurrent Neural Network

In [13]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM
from keras.optimizers import RMSprop

model = Sequential()
model.add(LSTM(128,input_shape = (SEQ_LEN,len(char)))) #long short term memory layer # 128 neurons, set to True
model.add(Dense(len(char))) #increase complexity
model.add(Activation("softmax")) # results add to one
model.compile(loss="categorical_crossentropy",optimizer=RMSprop(lr=0.01))
model.fit(x,y,batch_size=256,epochs=4)  #batch size : samples per gradient update, epoch: num iterations over entire x, y


E:\Users\sidmo\anaconda3\envs\tensor\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/4
1453/1453 [==============================] - 152s 101ms/step - loss: 1.9258
Epoch 2/4
1453/1453 [==============================] - 158s 109ms/step - loss: 1.5930
Epoch 3/4
1453/1453 [==============================] - 133s 91ms/step - loss: 1.5173
Epoch 4/4
1453/1453 [==============================] - 122s 84ms/step - loss: 1.4790


Helper to generate reasonable text from Keras Tutorial to check risk

In [14]:
def sample(predictions, temperature =1.0):
    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions)/temperature
    exp_preds = np.exp(predictions )
    predictions = exp_preds/ np.sum (exp_preds)
    probas = np.random.multinomial(1,predictions,1)
    return np.argmax(probas)

Generate Text

In [15]:
def text_gen(length,temperature):
    start_index = random.randint(0,len(ftext) - SEQ_LEN -1) # choose random starting position
    gen =''
    sentence = ftext[start_index:start_index +SEQ_LEN] # get sentence
    gen += sentence
    for i in range(length): # run loop to get characters
         x_pred = np.zeros((1,SEQ_LEN,len(char))) # predicted array
         for t, c in enumerate(sentence):
             x_pred[0,t,char_index[c]] = 1
         predictions = model.predict(x_pred, verbose=0)[0] #predict next char
         next_index = sample(predictions, temperature)
         next_character = index_char[next_index]
         gen += next_character
         sentence = sentence[1:] +next_character
    return gen

In [16]:
text_gen(300,0.2)

"e 'ever' last?\n\nKING RICHARD III:\nSweetly so so have the contraint that shall\nto the consuler and this son and the son\nTo stay the sending of the brother the house.\n\nCLARENCE:\nI am the death that the sense that the state\nThat the wars that will be the countervised\nTo the son that the way to the son and sets\nThat the state of the counter'd"